## Authentication

- Redhat identity management
- FreeIPA (Identity, Policy, Audit)
- NTP, DNS, LDAP, Kerberos, certs
- ipa-server-install
- need to have valid kerberos ticket for ipa

- https://www.freeipa.org/page/Quick_Start_Guide
- https://www.certdepot.net/rhel7-configure-freeipa-server/

- to create user accounts:
```
kinit admin
ipa user-add lisa
ipa passwd lisa
ipa user-find lisa
```

- authcofig tool to configure authentication

- Steps:

```
yum -y install ipa-server bind-dyndb-ldap
ipa-server-install
systemctl restart sshd # to obtain Kerberos credentials
kinit admin  # verify kerberos authentication
ipa user-find admin # verify ipa access
```

#### Kerberos tickets

to be able to manage IdM Server, you need to log in to IdM Domain to get a Kerberos ticket for the admin

- IMPORTANT, before doing anything, generate ticket: **```kinit admin```**
- show ticket validity: **```klist```**

Authconfig

- configuration files
```
/etc/sysconfig/authconfig
/etc/sssd/sssd.conf
/etc/krb5.cfg
/etc/nsswitch.conf
/etc/nslcd.conf  # old LDAP configuration file, now use /etc/sssd/sssd.conf instead
```

Exercise authconfig

- set up server2 as an IdM server

```
ssh server2

systemctl stop firewall # rembmer to start after
cat > /etc/hosts <<EOF
192.168.4.101 server1.example.com server1
192.168.4.102 server2.example.com server2 
EOF

yum -y install ipa-server bind nds-ldap bind-dyndb-ldap

ipa-server-install
    > yes to configure BIND
    > server2.example.com as host name
    > DNS forwarder yes
    > DNS forwarder: 8.8.8.8

systemclt restart sshd
knit admin
ip user-find admin
```

- create two users: lisa and lori, with the password: password

```
ipa user-add lisa
ipa passwd lisa
ipa user-find lisa
```

- configure your server as a Kerberos client to itself

```
ssh server1
cat > /etc/hosts <<EOF
192.168.4.101 server1.example.com server1
192.168.4.102 server2.example.com server2 
EOF

authconfig-tui
    > Use LDAP
    > Use Kerberos Authentication
    > ldap://server2.example.com
    > Kerberos settings: Realm: EXAMPLE.COM
    > use KDC to locate realm 

```

- test logging in as one of the use

## iSCSI

https://www.certdepot.net/rhel7-configure-iscsi-target-initiator-persistently/

iSCSI Configuration (Internet SCSI, Internet Small Computer Systems Interface)

- it provides block-level access to storage devices over a TCP/IP network
- iSCSI can be used to transmit data over local area networks (LANs), wide area networks (WANs), or the Internet and can enable location-independent data storage and retrieval
- much cheaper than fibre chanel, it uses ethernet cable
- it is a storage area network (SAN) protocol

#### Terminology

- **IQN**: iSCSI Qualified Name, to identify targets and initiators, it can be used in security
- Initiator: iSCSI client, identified by an IQN
- Target: the service on an iSCSI server that gives access to the backend services (SAN)
- ACL: Access Control List
- discovery: process where initiator find the targets configured on portal
- Portal: ip address and port use by target
- **LUN**: logical unit, the block devices shared through the target
- **TPG**: target portal group, collection of IP addresses and tcp ports to which iscsi target will listen

In [5]:
! cat /proc/partitions

major minor  #blocks  name

   8        0  234431064 sda
   8        1     204800 sda1
   8        2     256000 sda2
   8        3   61747200 sda3
   8        4  172222023 sda4
  11        0    1048575 sr0
 253        0   61745152 dm-0
 253        1   26214400 dm-1
 253        2    4063232 dm-2
 253        3   67108864 dm-3
 253        4   73400320 dm-4
 253        5   31457280 dm-5


In [6]:
# yum -y install lsscsi
!lsscsi

[0:0:0:0]    disk    ATA      ADATA SP550      3B5a  /dev/sda 
[1:0:0:0]    cd/dvd  HL-DT-ST DVD+-RW GU90N    A1B0  /dev/sr0 


#### configurations

- to install iscsi tools: ```yum install -y targetcli```

- commands to create new block, iscsi SAN (on server2)

```
targetcli
    > ls
    > cd backstores
    > block/ create block1 /dev/vgsan/lvsan1 # it creates block storage object block1 using /dev/vgsan/lvsan1
    > ls

    > cd iscsi
    > create iqn.2016-09.com.example.com:target1  # create target

    # to create acls
    > cd iqn.2016-09.com.example.com:target1
    > cd tpg1
    > acls create iqn.2016-09.com.example.com:server2

    # to create luns
    > luns/ create /backstores/block/block1

    # to create portal
    > portals/ create IPADDRESS

# it saves configuration to file /etc/target/saveconfig.json

# open firewall 3260 tcp

firewall-cmd --add-port=3260/tcp --permanent
fiewall-cmd --reload
systemctl enable target
systemctl start target
```

- commands to create iscsi Initiator (client, on server1)

```
yum install -y iscsi-initiator-utils

# set initiator name
vim /etc/iscsi/initiatorname.iscsi

# to find examples
man iscsiadm

iscsiadm --mode discoverydb --type sendtargets --portal TARGET_IPADDR --discover

iscsiadm --mode node --targetname TARGET_IQN --portal TARGET_IPADDR --login

yum -y install lsscsi
lsscsi  - will show new devices

# should show portal information file
ls /var/lib/iscsi/nodes

# important to logout
iscsiadm --mode node --targetname TARGET_IQN --portal TARGET_IPADDR --logout
```

- verifying the iscsi connection

```
# on client
iscsiadm -m session -P 3
iscsiadm -m node -P 3
iscsiadm -m discovery -P 3

# on target
targetcli
```

- Examples in doc: ```man targetcli```

- **IMPORTANT: to mount network device use ```_netdev``` option in fstab**

## System performance monitoring

SAR (System Activity Reporter)

- to have SAR you need to install **sysstat** package: `yum install -y sysstat`
- config file: `/etc/sysconfig/sysstat`
- run by cron:  `/etc/cron.d/sysstat`
- data is written to `/var/log/sa`

In [11]:
# network info
! LANG=C sar -n DEV | tail -n 10

10:30:07    virbr0-nic      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
10:30:07       wlp6s0    223.90    103.49    167.71     10.11      0.00      0.00      0.00      0.00
10:30:07       virbr0      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
10:30:07       enp7s0      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
10:30:07           lo      0.24      0.24      0.11      0.11      0.00      0.00      0.00      0.00
Average:    virbr0-nic      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
Average:       wlp6s0     54.39     25.57     40.21      2.60      0.00      0.00      0.00      0.00
Average:       virbr0      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
Average:       enp7s0      0.00      0.00      0.00      0.00      0.00      0.00      0.00      0.00
Average:           lo      0.11      0.11      0.40      0.40      0.00      0.0

In [ ]:
# set up sar to collect data every 5 minutes, display CPU usage and network usage

sar -P 0
sar -n DEV

In [ ]:
# use sar to generate an overview of the queue length and load averages at any given moment

sar -q

In [1]:
! cat /proc/meminfo

MemTotal:        3956064 kB
MemFree:           50564 kB
MemAvailable:     829564 kB
Buffers:           13052 kB
Cached:          1357624 kB
SwapCached:         3260 kB
Active:          1313124 kB
Inactive:        1173248 kB
Active(anon):     973016 kB
Inactive(anon):   833596 kB
Active(file):     340108 kB
Inactive(file):   339652 kB
Unevictable:         164 kB
Mlocked:             164 kB
SwapTotal:       4063228 kB
SwapFree:        4040792 kB
Dirty:               732 kB
Writeback:             0 kB
AnonPages:       1112856 kB
Mapped:          1953264 kB
Shmem:            690992 kB
Slab:             196188 kB
SReclaimable:     128616 kB
SUnreclaim:        67572 kB
KernelStack:        9424 kB
PageTables:        41820 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6041260 kB
Committed_AS:    6790944 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
HardwareCorrupted:     0 kB
AnonHugePages:   

In [2]:
! vmstat

procs -----------memory---------- ---swap-- -----io---- -system-- ------cpu-----
 r  b   swpd   free   buff  cache   si   so    bi    bo   in   cs us sy id wa st
 2  0  23000  38012  13952 1559876    0    1    56    40  335  608  1  2 97  0  0


## System Optimization Basics

In [12]:
! ls /proc/sys

abi  crypto  debug  dev  fs  kernel  net  sunrpc  vm


sysctl settings are defined through files in
- /usr/lib/sysctl.d/
- /run/sysctl.d/
- /etc/sysctl.d/

In [13]:
! sysctl -a

abi.vsyscall32 = 1
crypto.fips_enabled = 0
debug.exception-trace = 1
debug.kprobes-optimization = 1
dev.cdrom.autoclose = 1
dev.cdrom.autoeject = 0
dev.cdrom.check_media = 0
dev.cdrom.debug = 0
dev.cdrom.info = CD-ROM information, Id: cdrom.c 3.20 2003/12/17
dev.cdrom.info = 
dev.cdrom.info = drive name:		sr0
dev.cdrom.info = drive speed:		24
dev.cdrom.info = drive # of slots:	1
dev.cdrom.info = Can close tray:		1
dev.cdrom.info = Can open tray:		1
dev.cdrom.info = Can lock tray:		1
dev.cdrom.info = Can change speed:	1
dev.cdrom.info = Can select disk:	0
dev.cdrom.info = Can read multisession:	1
dev.cdrom.info = Can read MCN:		1
dev.cdrom.info = Reports media changed:	1
dev.cdrom.info = Can play audio:		1
dev.cdrom.info = Can write CD-R:		1
dev.cdrom.info = Can write CD-RW:	1
dev.cdrom.info = Can read DVD:		1
dev.cdrom.info = Can write DVD-R:	1
dev.cdrom.info = Can write DVD-RAM:	1
dev.cdrom.info = Can read MRW:		1
dev.cdrom.info = Can write MRW:		1
dev.cdrom.info = Can write RAM:		1
d

In [16]:
# turn off ICMP

!# echo 1 > /proc/sys/net/ipv4/icmp_echo_ignore_all = 0

# cat > /etc/sysctl.conf <<EOF
net.ipv4.icmp_echo_ignore_all = 0
EOF

sysctl: permission denied on key 'fs.protected_hardlinks'
sysctl: permission denied on key 'fs.protected_symlinks'
sysctl: permission denied on key 'kernel.cad_pid'
sysctl: permission denied on key 'kernel.usermodehelper.bset'
sysctl: permission denied on key 'kernel.usermodehelper.inheritable'
net.ipv4.icmp_echo_ignore_all = 0
net.ipv4.icmp_echo_ignore_broadcasts = 1
net.ipv4.icmp_errors_use_inbound_ifaddr = 0
net.ipv4.icmp_ignore_bogus_error_responses = 1
net.ipv4.icmp_msgs_burst = 50
net.ipv4.icmp_msgs_per_sec = 1000
net.ipv4.icmp_ratelimit = 1000
net.ipv4.icmp_ratemask = 6168
sysctl: permission denied on key 'net.ipv4.tcp_fastopen_key'
sysctl: permission denied on key 'net.ipv6.conf.all.stable_secret'
sysctl: permission denied on key 'net.ipv6.conf.default.stable_secret'
sysctl: permission denied on key 'net.ipv6.conf.enp7s0.stable_secret'
sysctl: permission denied on key 'net.ipv6.conf.lo.stable_secret'
sysctl: permission denied on key 'net.ipv6.conf.vboxnet0.stable_secret'
sysctl

## Logging

#### Connecting Journald to Rsyslog

In /etc/rsyslog.conf

`
$ModLoad imuxsock
$OmitLocalLogging off
$ModLoad omjournal     # sending to journal 
*.* :omjournal:        # om - output module, im - input module
`

In /etc/rsyslog.d/listend.conf

`
$SystemLogSocketName /run/systemd/journal/syslog
`

- configure server1 as a log server

```
vim /etc/rsyslog.conf
    > $ModLoad imuxsock   # provides support for local system logging (e.g. via logger command)
    > $OmitLocalLogging off
    > $ModLoad imjournal  # provides access to the systemd journal
   
vim /etc/rsyslog.d/listend.conf
    > $SystemLogSocketName /run/systemd/journal/syslog
```

- from server2 send all logs to server1
- https://devops.profitbricks.com/tutorials/configure-remote-logging-with-rsyslog/


```
vim /etc/rsyslog.conf
    > $ModLoad imtcp           # Provides TCP syslog reception
    > $InputTCPServerRun 514   # @@ for TCP, @ for UDP
    > *.* @@server1:514        # or local6.info @10.11.12.13:514 to send particular logs

```

- http://linux.byexamples.com/archives/412/syslog-sending-log-from-remote-servers-to-syslog-daemon/

- Facility defines the source of the log entries, what kind of services that send this logs
- Severity is the log levels that defines how critical of the log entries, from 0 – 7, 0 indicates the most critical and 7 is for debugging purpose

- `facility.severity         log_files`
- `user.notice              /var/log/user.notice`

## Networking

`man nmcli-examples`


#### Network Bridges

`yum install bridge-utils`

In [18]:
# ethernet bridge administration
! brctl show

bridge name	bridge id		STP enabled	interfaces
virbr0		8000.525400f67c4e	yes		virbr0-nic


In [22]:
! ip link show

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default 
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: enp7s0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc fq_codel state DOWN mode DEFAULT group default qlen 1000
    link/ether 34:e6:d7:50:ce:34 brd ff:ff:ff:ff:ff:ff
3: wlp6s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP mode DORMANT group default qlen 1000
    link/ether f4:06:69:57:e0:f9 brd ff:ff:ff:ff:ff:ff
4: virbr0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN mode DEFAULT group default qlen 1000
    link/ether 52:54:00:f6:7c:4e brd ff:ff:ff:ff:ff:ff
5: virbr0-nic: <BROADCAST,MULTICAST> mtu 1500 qdisc noqueue master virbr0 state DOWN mode DEFAULT group default qlen 500
    link/ether 52:54:00:f6:7c:4e brd ff:ff:ff:ff:ff:ff
6: vboxnet0: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN mode DEFAULT group default qlen 1000
    link/ether 0a:00:27:00:00:00 brd ff:ff:ff:ff:ff:ff
7

## Firewall

- configure service with the name myservice, accessing port: 2022

```
cp /usr/lib/firewalld/services/ssh.xml /etc/firewalld/services/myservice.xml
    > update port from 22 to 2022
    
vim /etc/ssh/sshd_config 
    > change listening port to 2022
    
semanage port -l | grep ssh
semanage port -a -t ssh_port_t -p tcp 2022

systemctl restart sshd

firewall-cmd --get-services
firewall-cmd --add-service=myservice --permanent
firewall-cmd --reload
```

- on server1 configure port forwarding to make sure the SSHD process is available on port 2222

`
firewall-cmd --permanent --zone=public --add-forward-port=port=2222:proto=tcp:toport=2022
`

## Apache



- main configuration: `/etc/httpd/conf/httpd.conf`
- to check configuration syntax use: `httpd -t`
- modules: `/etc/httpd/conf.modules.d`

###### **IMPORTANT**:to have access to http manuall

`yum install -y httpd-manual` and to run `firefox http://localhost/manual`

`yum install -y elinks` 

`firewall-cmd --permanent --add-service=http --add-service=https`

- to check SELinux: ` ls -Zd /var/www/html`
- to set SELinux on dir: `semanage fcontext -a -t httpd_sys_content_t '/web(/.*)?' && restorecon -R -v /web`

##### to set access for web developers to web dir

```
groupadd webdev
setfacl -R -m g:webdev:rwX /web
setfacl -R -m d:g:webdev:rwX /web
getfacl /web
```

##### to quickly check http logs

`systemctl status -l httpd` or `journalctl UNIT=httpd.service`

#### Http authentication

`htpasswd -c /usr/local/apache/passwd/passwords username`


#### TLS Protected Web Sites

```
yum install -y crypto-utils mod_ssl
genkey server1.example.com   # it creates 2 keys to encrypt SSL trafic, keys are in /etc/pki/tls

vim /etc/httpd/conf.d/ssl.conf
    > SSLCertificateFile    /etc/pki/tls/certs/server1.example.com.crt
    > SSLCertificateKeyFile /etc/pki/tls/private/server1.example.com.key
    
systemctl restart httpd
```

## DNS

### Unbound

- for cache only DNS, use unbound instead of BIND: `yum install -y unbound`
- configuration in /etc/unbound/unbound.conf

##### how to set up cache only dns

```
vim /etc/unbound/unbound.conf
    > interface: 0.0.0.0
    > access-control: 0.0.0.0/0 allow
    > forward-zone:
        name: "."
        forward-addr: 8.8.8.8
       
unbound-checkconf   # check for syntax error
systemctl restart unbound

firewall-cmd --permanent --add-service=dns
firewall-cmd --reload
```

###### how to work with dns cache dumps

```
unbound-control dump_cache > cachefile
```

##### how to clean dns db

```
unbound-control flush google.com
```

##### load from cache

```
unbound-control load_cache cachefile
```

### Bind

##### how to set up cache only dns

```
yum install -y bind bind-utils

vim /etc/named.conf
    > listen-on port 53 { any; }; 
    > allow-query: { any; };
    > dnssec-validation: no;
    
named-checkconf

firewall-cmd --permanent --add-service=dns && firewall-cmd --reload

systemctl enable named
systemctl start named
```

##### to test local dns server

```
nslookup google.com 127.0.0.1
digg @127.0.0.1 google.com
```

In [1]:
! dig -x 172.217.16.206


; <<>> DiG 9.10.3-P3-RedHat-9.10.3-10.P3.fc23 <<>> -x 172.217.16.206
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 7898
;; flags: qr rd ra; QUERY: 1, ANSWER: 2, AUTHORITY: 0, ADDITIONAL: 0

;; QUESTION SECTION:
;206.16.217.172.in-addr.arpa.	IN	PTR

;; ANSWER SECTION:
206.16.217.172.in-addr.arpa. 8575 IN	PTR	fra16s08-in-f206.1e100.net.
206.16.217.172.in-addr.arpa. 8575 IN	PTR	fra16s08-in-f14.1e100.net.

;; Query time: 7 msec
;; SERVER: 188.121.31.151#53(188.121.31.151)
;; WHEN: Fri Sep 09 20:38:44 CEST 2016
;; MSG SIZE  rcvd: 115



## MariaDB

##### Configuration

```
yum install -y mariadb mariadb-libs mariadb-test
mysql_secure_installation

vim /etc/my.cnf

mysql -u root -h localhost -p
```

#### Backups

######  logical backup - SQL dump

```
mysqldump -u root -p --all-databases > dump.sql  # backup
mysqldump -u root -p < dump.sql  # restoring backup

```

###### physical backup (creating DB snapshot in LVM)

```
# find data directory to backup
# let's assume that current DB is on /dev/vgdata/lvmysql disk

mysqladmin -u root -p variables | grep datadir  #

# check disk space in LVM
vgs 

# Freeze database to be sure there are no modifications
mysql -u root -p
    > FLUSH TABLES WITH READ LOCK;
    > UNLOCK TABLES;                 # IMPORTANT: unlock AFTER creating snapshot, see below!!!!
    
# from another terminal create an LVM snapshot
# you can take 10% of disk size to create snapshot, so 2GB of snapshot if we have 20GB database

lvcreate -L 2G --snapshot -n lvmysql-snapshot /dev/vgdata/lvmysql

mkdir /mnt/snapshot
mount /dev/vgdata/lvmysql-snapshot /mnt/snapshot
tar -czvf /root/mysql-backup.tar.gz /mnt/snapshot
umount /mnt/snapshot
lvremove /dev/vgdata/lvmysql-snapshot
```

- to restore physical backup

```
systemctl stop mariadb
rm /var/lib/mysql/* -rf
tar xzvf /root/mysql-backup.tar.gz -C /
```

## Time services

##### Chronyc
- use timedatectl to set time on server1 one hour ahead

`timedatectl time-set 16:00`

- configure NTP synchronization on server1 and server2 to use NTP peers only; both should use on another as peers

`timedatectl set-ntp on`

- monitor time synchronization progress

`
chronyc sources -v
chronyc tracking
`

##### NTP

```
systemctl start ntpd
ntpq -p
ntpstat

vim /etc/ntp.conf
    > comment out all servers
    > server 127.127.1.0  # add local server
    
firewall-cmd --permanent --add-service=ntp

```

## NFSv4

- listens on 2049 port
- for nfs 2 and 3, remember to install rpcbind (https://access.redhat.com/documentation/en-US/Red_Hat_Enterprise_Linux/7/html/Storage_Administration_Guide/s2-nfs-methodology-portmap.html)

- in /etc/fstab mount NFS with `_netdev` option
- check /var/lib/nfs/etab
- man exportfs

- important options in /etc/exports file

```
root_squash    -> map root UID/GID to anonymous UID/GID (nobody/nogroup); this is the default
all_squash     -> map all UIDs/GIDs to anonymous UID/GID (nobody/nogroup)
no_root_squash -> do not map root (nor any other) UID/GID to anonymous UID/GID (nobody/nogroup)

```

##### How to set up basic NFS share

```
vim /etc/exports
    > /share *(rw,all_squash)
    
mkdir /share
chown nfsnobody /share
systemctl start nfs-server

exportfs -r  # updates all shares, no nfs restart is needed

showmount -e localhost
```

##### Accessing NFS share - client

```
yum install nfs-utils
showmount -e server2
mkdir /nfs
mount server2:/share /nfs
mount | grep nfs
```

##### set up shared directory for group

- change directory owner to **nfsnobody**
-  By default, NFS shares change the root user to user nfsnobody, an unprivileged user account. This way all root-created files are owned by user nfsnobody, which prevents uploading of programs with the setuid bit set.
If no_root_squash is used, remote root users will be able to change any file on the shared file system and leave trojaned applications for other users to inadvertently execute.

- do the same on client and server
```
groupadd -g 7654 nfsdatagrp
usermod -G nfsdatagrp user1
usermod -G nfsdatagrp user2

mkdir /nfsdata
chown nfsnobody:nfsdatagrp /nfsdata
chmod 2770 /nfsdata


```

#### Using Kerberos to Control Access to NFS Network

- SELinux boolean is required: **nfsd_anon_write**
- nfs-server and nfs-secure-server must be installed!!!
- /etc/krb5.keytab is needed - it includes host principal, nfs principal
- Kerberos principals are: accounts and keys for system
- `klist -k` to view the contents of a keytab

##### setting up kerberized NFS server


```
# setting up kerberized NFS

yum install nfs-secure-server nfs-server nfs-utils

# copy krb5.keytab to /etc/krb5.keytab

vim /etc/sysconfig/nfs
    > RCNFSDARGS="-V 4.2"

systemctl start nfs-server
systemctl start nfs-secure-server

systemctl enable nfs-server
systemctl enable nfs-secure-server

mkdir /secureshare

# set up SELinux context for directory
getsebool -a | grep nfs_export   # to check if options are on or off

setsebool -P nfs_export_all_ro=1
setsebool -P nfs_export_all_rw=1


vim /etc/exports
    > /secureshare *(sec=krb5p,rw)
    
exportfs -r

firewall-cmd --permanent --add-service=nfs
fireall-cmd --reload
```

##### setting up kerberized NFS client

```
systemctl enable nfs-secure
systemctl start nfs-secure

mount -o sec=krb5p server2:/secureshare /mnt
```


## Samba


